Modify the Gradient Boosting scratch code in our lecture such that:
- Notice that we are still using max_depth = 1.  Attempt to tweak min_samples_split, max_depth for the regression and see whether we can achieve better mse on our boston data
- Notice that we only write scratch code for gradient boosting for regression, add some code so that it also works for binary classification.  Load the breast cancer data from sklearn and see that it works.
- Further change the code so that it works for multiclass classification.  Load the digits data from sklearn and see that it works
- Put everything into class

In [1]:
from scipy.special import expit
from sklearn.tree import DecisionTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
import numpy as np

In [6]:
class GradientBoosting:
    def __init__(self, S=5, learning_rate=1, max_depth = 1, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4):
        self.S = S
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.regression=regression

        #initialize regression trees
        tree_params = {'max_depth': self.max_depth,
                      'min_samples_split': self.min_samples_split}
        self.models = [DecisionTreeRegressor(**tree_params) for _ in range(S)]         #create tree
        first_model = DummyRegressor(strategy='mean')  #use mean / stump
        if regression == True:
            # if GBregressor use Regressor as the first model
            first_model = DummyRegressor(strategy='mean')
        else :
            # if GBclassifier use Classifier as the first model
            first_model = DummyClassifier(strategy='most_frequent')
            
        self.models.insert(0, first_model) #add mean y to models


    def grad(self, y, h):  #div loss
        return y - h 

    def predict(self, X, models=None, with_argmax=True):
        if models is None:
            models = self.models
        h0 = models[0].predict(X)  #first use the dummy model
        boosting = sum(self.learning_rate * model.predict(X) for model in models[1:])
        yhat = h0 + boosting
        
        if not self.regression:
            #turn into probability using softmax
            yhat = np.exp(yhat) / np.sum(np.exp(yhat), axis=1, keepdims=True)
            if with_argmax:
                yhat = np.argmax(yhat, axis=1)
        return yhat
    
    def fit(self, X, y):  #<----X_train
        
        #fit the first model
        self.models[0].fit(X, y)
        
        for i in range(self.S):
            #predict
            yhat = self.predict(X, self.models[:i+1], with_argmax=False)
            
            #get the gradient
            gradient = self.grad(y, yhat)
            
            #fit the next model with gradient
            self.models[i+1].fit(X, gradient)

In [7]:
# Regression
from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                        test_size=0.3, random_state=42)

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 1, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
print(X_test[1])
print(yhat[1])
#print metrics
print("MSE max_depth =1: ", mean_squared_error(y_test, yhat))

#=====SKlearn========
#Compare to sklearn: ls is the same as our mse
n_estimators = 200
sklearn_model = GradientBoostingRegressor(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1,
    loss='ls'
)

yhat_sk = sklearn_model.fit(X_train, y_train).predict(X_test)
print("Sklearn MSE max_depth= 1: ", mean_squared_error(y_test, yhat_sk))

[5.6440e-02 4.0000e+01 6.4100e+00 1.0000e+00 4.4700e-01 6.7580e+00
 3.2900e+01 4.0776e+00 4.0000e+00 2.5400e+02 1.7600e+01 3.9690e+02
 3.5300e+00]
33.04343085396104
MSE max_depth =1:  12.945557601580582
Sklearn MSE max_depth= 1:  12.945557601580587


In [8]:
# Attempt to tweak min_samples_split, max_depth for the regression and see whether we can achieve better mse on our boston data
model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 1, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =1,min_sample= 2: ", mean_squared_error(y_test, yhat))

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 3, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =3,min_sample= 2: ", mean_squared_error(y_test, yhat))


model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 5, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =5,min_sample= 2: ", mean_squared_error(y_test, yhat))

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 7, 
                 min_samples_split = 2,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =7,min_sample= 2: ", mean_squared_error(y_test, yhat))


model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 1, 
                 min_samples_split = 4,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =1, min_sample= 4 : ", mean_squared_error(y_test, yhat))

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 7, 
                 min_samples_split = 14,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =7, min_sample= 14 : ", mean_squared_error(y_test, yhat))

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 7, 
                 min_samples_split = 16,
                 regression=True, tol=1e-4)                     # set regresssion here
model.fit(X_train, y_train)
yhat = model.predict(X_test)
#print metrics
print("MSE max_depth =3, min_sample= 16 : ", mean_squared_error(y_test, yhat))

MSE max_depth =1,min_sample= 2:  12.945557601580582
MSE max_depth =3,min_sample= 2:  8.15206238292054
MSE max_depth =5,min_sample= 2:  7.936777040494664
MSE max_depth =7,min_sample= 2:  7.321226464228971
MSE max_depth =1, min_sample= 4 :  12.945557601580582
MSE max_depth =7, min_sample= 14 :  6.995328028543036
MSE max_depth =3, min_sample= 16 :  8.39413341725854


Change max_depth and min_sample at proper number can achieve better mse on our boston data.

In this experiment max_depth =7, min_sample= 14 are the best number that make mse is the least value.



In [9]:
# Binary classification
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X, y = load_breast_cancer(return_X_y=True)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=42)
y_train_encoded = np.zeros((y_train.shape[0], len(set(y))))
for each_class in range(len(set(y))):
    cond = y_train==each_class
    y_train_encoded[np.where(cond), each_class] = 1

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 3, 
                 min_samples_split = 2,
                 regression=False)    # set regresssion here
model.fit(X_train, y_train_encoded)
yhat = model.predict(X_test)
print(X_test[1])
print(yhat[1])
# #print metrics
print("Our accuracy: ", accuracy_score(y_test, yhat))



#=====SKlearn========
#Compare to sklearn: ls is the same as our accuracy
n_estimators = 200
sklearn_model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1
)

yhat_sk = sklearn_model.fit(X_train, y_train).predict(X_test)
print("Sklearn accuracy: ", accuracy_score(y_test, yhat_sk))

[1.894e+01 2.131e+01 1.236e+02 1.130e+03 9.009e-02 1.029e-01 1.080e-01
 7.951e-02 1.582e-01 5.461e-02 7.888e-01 7.975e-01 5.486e+00 9.605e+01
 4.444e-03 1.652e-02 2.269e-02 1.370e-02 1.386e-02 1.698e-03 2.486e+01
 2.658e+01 1.659e+02 1.866e+03 1.193e-01 2.336e-01 2.687e-01 1.789e-01
 2.551e-01 6.589e-02]
0
Our accuracy:  0.9707602339181286
Sklearn accuracy:  0.9649122807017544


In [10]:
# Multiclass classification
# from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier

X, y = load_digits(return_X_y=True)

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=42)
y_train_encoded = np.zeros((y_train.shape[0], len(set(y))))

#encode method
for each_class in range(len(set(y))):
    cond = y_train==each_class
    y_train_encoded[np.where(cond), each_class] = 1

model = GradientBoosting(S=200, learning_rate=0.1, max_depth = 1, 
                 min_samples_split = 2,
                 regression=False)  # set regresssion here
model.fit(X_train, y_train_encoded)
yhat = model.predict(X_test)
print(X_test[1])
print(yhat[1])
# #print metrics
print("Our accuracy: ", accuracy_score(y_test, yhat))

#=====SKlearn========
#Compare to sklearn: ls is the same as our accuracy
n_estimators = 200
sklearn_model = GradientBoostingClassifier(
    n_estimators=n_estimators,
    learning_rate = 0.1,
    max_depth=1
)

yhat_sk = sklearn_model.fit(X_train, y_train).predict(X_test)
print("Sklearn accuracy: ", accuracy_score(y_test, yhat_sk))

[ 0.  0. 11. 16.  8.  0.  0.  0.  0.  6. 16. 11. 13.  9.  0.  0.  0.  7.
 16.  0.  9. 16.  0.  0.  0.  2. 15. 12. 16. 16.  3.  0.  0.  0.  5.  7.
  7. 16.  4.  0.  0.  0.  0.  0.  5. 16.  5.  0.  0.  0.  3.  7. 16. 11.
  0.  0.  0.  0. 13. 16. 11.  1.  0.  0.]
9
Our accuracy:  0.8055555555555556
Sklearn accuracy:  0.9481481481481482
